<a href="https://colab.research.google.com/github/elsebasantiago/Algoritmos_optimizacion_VIU/blob/main/Trabajo_Practico_Problema_del_doblaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico
# Sebastián Santiago
Link repositorio github
https://github.com/elsebasantiago/Algoritmos_optimizacion_VIU

#1. Introducción
### En el presente notebook se entrega el trabajo práctivo en donde se busca obtener el mejor resultado para el problema "Problema 1. Organizar sesiones de doblaje(I)".

*  Consigna: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible

* Número de actores: 10
* Número de tomas : 30
* Actores/Tomas : https://bit.ly/36D8IuK


### Se adjunta junto con la entrega del notebook 2 archivos:

1.   Datos problema doblaje.csv: Contiene la información entregada del problema en formato CSV.
2.   Datos problema doblaje-Analisis.xls: Contiene un excel con la validación manual de varios de resultados arrojados en las distintas aproximaciones de búsqueda de una solución.

> En las secciones donde los resultados sean deterministas, se hace referencia al libro en el excel donde se encuentra la verificación.







In [92]:
import os
import pandas as pd
import numpy as np
from collections import deque
import random

ruta = os.path.join("" ,"Datos problema doblaje.csv")
datos = pd.read_csv(ruta, sep=',')
datos = datos.drop(['Unnamed: 11'], axis =1)
tomas_actores = np.array(datos[['1', '2','3','4', '5', '6', '7','8','9','10']])

##Funciones Básicas

In [80]:
# generamos la fucnión de cálculo del costo
def costo_tomas(toma_origen, toma_destino):
  v_costo = toma_origen - toma_destino
  v_costo[v_costo > 0] = 0
  return sum(pow(element,2) for element in v_costo)


#Matriz de distancia entre tomas. Indica cuantos actores de diferencia tiene
# cada toma en la lista original (lista_tomas) versus una toma en particular
def matriz_distancia_toma(toma , lista_tomas):

  #Obtenemos la diferencia
  distancia_tomas = lista_tomas - toma

  #Los actores con valores negativos indican que están considerados en la toma
  # de comparación  por tanto, no los consideramos (igualamos a cero)
  # A su vez, se anulan los actores que participan en ambas tomas
  mask = distancia_tomas < 0
  distancia_tomas[mask] = 0

  #la suma de los actores restantes es la distancia entre las tomas
  return np.sum(distancia_tomas, axis = 1)

#Obtendremos el costo adicional que genera incorporar una nueva toma
def costo_adicional_toma (toma1, toma2):

  #creamos un vector diferencia entre tomas
  v_dif_tomas = toma2 - toma1

  #quitamos los valores negativos
  mask = v_dif_tomas <0
  v_dif_tomas[mask] = 0

  #sumamos los valores restantes para obtener el costo adicional
  #que incorporó la nueva toma
  return np.sum(v_dif_tomas)


#consideramos un actor mas a la lista de actores del día
def adiciona_actor(toma1, toma2):
  #obtenemos un vector que será la suma de los actores que participan
  #en ambas tomas
  v_sum_tomas = toma1 + toma2

  #normalizamos en 1 las posiciones de actores que coinciden
  mask = v_sum_tomas > 1
  v_sum_tomas[mask] = 1

  return v_sum_tomas

# Función que crea na solución aleatoria
def crea_solucion_aleatoria(n):
  return np.array(random.sample(range(1, n+1), n))



In [3]:
# creamos una función que devuelve los días, tomas y costos
def crear_dias_filmacion(lista_tomas, actores_en_toma):

  dias_tomas = np.zeros((len(lista_tomas),3)).astype(int)
  actores_toma_previa = np.zeros(actores_en_toma.shape[1])

  #Creamos una lista tipo fifo para ir recorriendo las tomas en la secuencia entregada
  lista_tomas_fifo = deque(lista_tomas.tolist())
  costo_total=0

  dia = 1
  j = 1
  i = 0

  while lista_tomas_fifo:

    # obtenemos el número de la primera toma
    toma = lista_tomas_fifo.popleft()
    #n_toma = lista_tomas[i]

    # obtenemos los actores que participan en la toma
    actores_toma_actual = actores_en_toma[toma-1]

    # Calculamos el costo de la toma, según la cantidad de actores
    costo_toma = np.sum(actores_toma_actual)
    costo_total += costo_toma

    # Matríz (día de toma, número de toma)z
    dias_tomas[i]= [dia, toma, costo_toma]

    #print(f'día = {dia} nro_toma = {toma} toma actual = {actores_toma_actual} costo = {costo_toma}')

    j=1

    while j < 6 and lista_tomas_fifo:

      #Tomamos la siguiente toma en la lista
      toma = lista_tomas_fifo.popleft()

      #Obtenemos los actores que deben participar en la siguiente toma en la lista
      actores_toma_siguiente = actores_en_toma[toma-1]

      #Calculamos el costo adiciona que conlleva considerar la nueva toma
      costo_toma = costo_adicional_toma(actores_toma_actual, actores_toma_siguiente)

      #Almacenamos el día, número de toma y costo adicional de la nueva toma
      dias_tomas[i+j]= [dia, toma, costo_toma]

      #print(f'dia = {dia} nro_toma = {toma} toma actual = {actores_toma_siguiente} costo = {costo_toma}')

      #Incluimos los nuevos actores a la lista de actores considerados para el día
      actores_toma_actual = adiciona_actor(actores_toma_actual,actores_toma_siguiente)
      costo_total += costo_toma
      j+=1

    i+=j
    dia +=1
  #print (f'costo total = {costo_total}')
  return dias_tomas


## 2. Prueba de diferentes soluciones manuales

#### En una primera instancia obtendremos posibles soluciones manuales considerando diferentes ordenamiento de las tomas, y así obtener diferentes líneas bases de resultados con los cuales comparar.

#### Luego veremos si con un aproximación metaheurística podríamos obtener mejores resultados


In [19]:
#Calculamos el costo de filmar las tomas en el orden presentado (de menor a mayor)
# Libro Excel: "Orden Original"

ds = datos.sort_values(by='Toma', ascending=True)
tomas = np.array(ds['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1  1  5]
 [ 1  2  0]
 [ 1  3  1]
 [ 1  4  1]
 [ 1  5  0]
 [ 1  6  0]
 [ 2  7  4]
 [ 2  8  1]
 [ 2  9  0]
 [ 2 10  1]
 [ 2 11  2]
 [ 2 12  0]
 [ 3 13  3]
 [ 3 14  2]
 [ 3 15  2]
 [ 3 16  1]
 [ 3 17  0]
 [ 3 18  0]
 [ 4 19  2]
 [ 4 20  2]
 [ 4 21  2]
 [ 4 22  1]
 [ 4 23  0]
 [ 4 24  0]
 [ 5 25  4]
 [ 5 26  3]
 [ 5 27  0]
 [ 5 28  0]
 [ 5 29  1]
 [ 5 30  0]]
solucion = 38


In [94]:
#Calculamos el costo de filmar las tomas en el orden inverso
#al presentado (de mayor a menor)
# Libro Excel: "Orden Inverso Original"

ds =datos.sort_values(by='Toma', ascending=False)
tomas = np.array(ds['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1 30  2]
 [ 1 29  2]
 [ 1 28  0]
 [ 1 27  0]
 [ 1 26  2]
 [ 1 25  2]
 [ 2 24  2]
 [ 2 23  1]
 [ 2 22  2]
 [ 2 21  1]
 [ 2 20  1]
 [ 2 19  0]
 [ 3 18  2]
 [ 3 17  1]
 [ 3 16  2]
 [ 3 15  2]
 [ 3 14  0]
 [ 3 13  1]
 [ 4 12  5]
 [ 4 11  2]
 [ 4 10  1]
 [ 4  9  0]
 [ 4  8  0]
 [ 4  7  0]
 [ 5  6  4]
 [ 5  5  1]
 [ 5  4  1]
 [ 5  3  0]
 [ 5  2  1]
 [ 5  1  0]]
solucion = 38


In [22]:
#Calculamos el costo de filmar las tomas ordenándolas primero con las
#escenas que demanan mayor cantidad de acotores y avanzando en orden decreciente
# Libro Excel: "Orden Total Descendente"

ds = datos.sort_values(by='Total', ascending=False)
ds = ds.drop(['Total'], axis =1)

tomas = np.array(ds['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1  1  5]
 [ 1 11  1]
 [ 1 12  1]
 [ 1  4  1]
 [ 1 10  1]
 [ 1 22  0]
 [ 2 26  4]
 [ 2 20  1]
 [ 2 25  2]
 [ 2  7  0]
 [ 2  6  0]
 [ 2  9  0]
 [ 3  3  3]
 [ 3 13  2]
 [ 3  5  1]
 [ 3 15  0]
 [ 3  2  1]
 [ 3 29  1]
 [ 4  8  3]
 [ 4 14  1]
 [ 4 24  0]
 [ 4 28  1]
 [ 4 27  1]
 [ 4 16  1]
 [ 5 23  2]
 [ 5 21  2]
 [ 5 19  0]
 [ 5 18  0]
 [ 5 17  0]
 [ 5 30  1]]
solucion = 36


In [96]:
#Calculamos el costo de filmar las tomas ordenándolas primero con las
#escenas que demanan menor cantidad de acotores y avanzando en orden ascendente
# Libro Excel: "Orden Total Ascendente"

ds = datos.sort_values(by='Total', ascending=True)
ds = ds.drop(['Total'], axis =1)

tomas = np.array(ds['Toma'])

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1 30  2]
 [ 1 28  0]
 [ 1 27  1]
 [ 1 24  2]
 [ 1 23  0]
 [ 1 21  1]
 [ 2 19  2]
 [ 2 18  1]
 [ 2 17  0]
 [ 2 16  2]
 [ 2 29  1]
 [ 2 14  0]
 [ 3 15  3]
 [ 3  2  3]
 [ 3  9  0]
 [ 3  8  1]
 [ 3 13  0]
 [ 3  5  1]
 [ 4  3  3]
 [ 4 22  3]
 [ 4 10  2]
 [ 4  7  0]
 [ 4 20  0]
 [ 4  6  0]
 [ 5  4  4]
 [ 5 25  2]
 [ 5 26  3]
 [ 5 12  1]
 [ 5 11  0]
 [ 5  1  0]]
solucion = 38


#### Notas:


*   Mejor resultado obtenido: 36
*   El mejor resultado se obtuvo ordenando las escensas primero aquellas que tienen mayor participación de actores y avanzando en orden descendente.



# 3. Procedimiéntos Metaheurísticos
#### En la siguiente sección aplicaremos distíntas técnicas metaheurísticas buscando obtener soluciones que mejoren el resultado obtenido hasta ahora (36)

## Búsqueda Aleatoria

In [81]:
# Solución con búsqueda aleatoria
def busqueda_aleatoria(actores_en_toma):

  # definimos a N como el número de iteraciones
  n = 1000
  mejor_costo = 10e100
  mejor_solucion = []
  nt= len(actores_en_toma)

  while n > 0:

    lista_tomas = crea_solucion_aleatoria(nt)
    solucion = crear_dias_filmacion(lista_tomas, actores_en_toma)
    costo_actual = np.sum(solucion[:,2], axis = 0)

    if costo_actual < mejor_costo:
      mejor_solucion = solucion
      costo_actual = mejor_costo

    n-=1

  return mejor_solucion


In [84]:
# Invocamos la función de busqueda aleatoria buscando otbener buenas soluciones
tomas = crea_solucion_aleatoria(len(tomas_actores))

solucion = busqueda_aleatoria( tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


[[ 1 24  2]
 [ 1  9  3]
 [ 1 20  1]
 [ 1 12  0]
 [ 1 18  0]
 [ 1 27  0]
 [ 2 15  3]
 [ 2  6  2]
 [ 2  1  1]
 [ 2  5  1]
 [ 2  8  1]
 [ 2 14  0]
 [ 3 26  4]
 [ 3 29  1]
 [ 3 10  1]
 [ 3  7  1]
 [ 3 30  0]
 [ 3 17  0]
 [ 4 11  5]
 [ 4 19  0]
 [ 4 25  2]
 [ 4 13  0]
 [ 4  3  1]
 [ 4 23  0]
 [ 5 28  2]
 [ 5 16  1]
 [ 5 21  2]
 [ 5 22  2]
 [ 5  4  1]
 [ 5  2  1]]
solucion = 38


##Búsqueda local

In [43]:
# Generamos solución vecina

def genera_vecina(lista_tomas,actores_en_toma):
  #Generador de soluciones vecinas mediante 2-opt

  mejor_solucion = []
  mejor_costo = 10e100

  for i in range(1,len(lista_tomas)-1):
    for j in range(i+1, len(lista_tomas)-1):

      # intercalamos posiciones
      lista_tomas[i], lista_tomas[j] =  lista_tomas[j] , lista_tomas[i]

      #Se evalua la nueva solución
      solucion = crear_dias_filmacion(lista_tomas, actores_en_toma)
      costo_actual = np.sum(solucion[:,2], axis = 0)

      # Nos quedamos con la mejor solución
      if costo_actual < mejor_costo:
        mejor_solucion = solucion
        costo_actual = mejor_costo

  return mejor_solucion


In [45]:
# Solución con búsqueda local
def busqueda_local(actores_en_toma):

  solucion = busqueda_aleatoria(actores_en_toma)
  lista_tomas = solucion[:,1]
  mejor_costo = np.sum(solucion[:,2], axis = 0)
  mejor_solucion = solucion

  print(f'Solucion inicial = {lista_tomas}')
  print(f'costo inicial = {mejor_costo}')
  i = 50

  while i >0 :
    i-=1
    solucion = genera_vecina(lista_tomas,actores_en_toma)
    costo_tomas_vecina = np.sum(solucion[:,2], axis = 0)

    if costo_tomas_vecina < mejor_costo:
      mejor_solucion = solucion
      lista_tomas = solucion[:,1]
      mejor_costo = costo_tomas_vecina

  print(f'Costo final = {mejor_costo}')
  return mejor_solucion


solucion = busqueda_local(tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

Solucion inicial = [10 19 15 20 25 18 13 21  3  9 22 14 29 27 12  4  2  1  5 16 23  6 28 17
 11  7 30 24 26  8]
costo inicial = 40
Costo final = 39
[[ 1 10  4]
 [ 1 19  2]
 [ 1 15  0]
 [ 1 20  1]
 [ 1 25  0]
 [ 1 18  1]
 [ 2 13  2]
 [ 2 21  1]
 [ 2  3  2]
 [ 2  9  0]
 [ 2 22  1]
 [ 2 14  1]
 [ 3 29  5]
 [ 3 27  0]
 [ 3 12  2]
 [ 3  4  1]
 [ 3  2  0]
 [ 3  1  0]
 [ 4  5  3]
 [ 4 16  2]
 [ 4 23  0]
 [ 4  6  2]
 [ 4 28  1]
 [ 4 17  0]
 [ 5 11  2]
 [ 5  7  4]
 [ 5 30  1]
 [ 5 24  1]
 [ 5 26  0]
 [ 5  8  0]]
solucion = 39


## Solución Greedy

In [47]:
# Solución greedy
# busca la mejor solución obteniendo la mejor toma siguiente por medio
# de un algoritmo voraa, con lo cual escoje la siguiente toma con la menor
# distancia, es decir, con la menor cantidad de actores adicionales
# Libro Excel: "Greedy"

def crea_lista_greedy(lista_tomas, actores_en_toma):

  lista_tomas_disponibles = list(lista_tomas)
  resultado_tomas =[]

  toma = lista_tomas[0]
  lista_tomas_disponibles.remove(toma)
  actores_toma_actual = actores_en_toma[toma-1]
  m_distancia_a_tomas = matriz_distancia_toma(actores_toma_actual, actores_en_toma)
  #print(f'm_distancia_a_tomas = {m_distancia_a_tomas}')

  resultado_tomas.append(toma)
  m_distancia_a_tomas[toma-1] = 1000

  while len(lista_tomas_disponibles) > 0:
    toma_dist_menor = np.argmin(m_distancia_a_tomas) + 1

    if toma_dist_menor in lista_tomas_disponibles:
      resultado_tomas.append(toma_dist_menor)
      lista_tomas_disponibles.remove(toma_dist_menor)
      actores_toma_actual = adiciona_actor(actores_toma_actual,actores_en_toma[toma_dist_menor-1])
      m_distancia_a_tomas = matriz_distancia_toma(actores_toma_actual, actores_en_toma)
    else:
      m_distancia_a_tomas[toma_dist_menor-1] = 1000

  return np.array(resultado_tomas)


In [51]:
# Obtenemos un resultado por medio de un algoritmo greedy, partiendo de la lísta
# de tomas y actores en el orden original.

tomas_original = np.array(datos['Toma'])

tomas = crea_lista_greedy(tomas_original, tomas_actores)

solucion = crear_dias_filmacion(tomas, tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

[[ 1  1  5]
 [ 1  2  0]
 [ 1  6  0]
 [ 1  7  0]
 [ 1  9  0]
 [ 1 13  0]
 [ 2 17  2]
 [ 2 19  0]
 [ 2 20  2]
 [ 2 22  1]
 [ 2 23  0]
 [ 2 27  0]
 [ 3 28  2]
 [ 3 30  0]
 [ 3  3  3]
 [ 3 15  0]
 [ 3  4  1]
 [ 3  5  0]
 [ 4 11  5]
 [ 4  8  1]
 [ 4 12  1]
 [ 4 14  0]
 [ 4 18  0]
 [ 4 21  0]
 [ 5 24  2]
 [ 5 29  2]
 [ 5 10  2]
 [ 5 26  0]
 [ 5 16  2]
 [ 5 25  0]]
solucion = 31


In [85]:
# Solución greedy Multiarranque
# busca la mejor solución comenzando desde distintas
# configuraciones de órdenes de toma y por cada una busca la mejor toma por medio
# de un algoritmo voraz obteniendo la siguiente toma con la menor distancia, es decir,
# con la menor cantidad de actores adicionales
def solucion_greedy_multiarranque(actores_en_toma):

  lista_tomas  = []
  mejor_costo = 10e100
  mejor_solucion = []
  nt = len(actores_en_toma)

  n = 1000

  for i in range(n):
    tomas_multiarranque = crea_solucion_aleatoria(nt)
    lista_tomas = crea_lista_greedy(tomas_multiarranque, actores_en_toma)
    solucion = crear_dias_filmacion(lista_tomas, tomas_actores)
    costo_nueva_solucion = np.sum(solucion[:,2], axis = 0)

    if costo_nueva_solucion < mejor_costo:
      mejor_solucion = solucion
      lista_tomas = solucion[:,1]
      mejor_costo = costo_nueva_solucion
      print(f'Iteracion = {i} Costo = {mejor_costo}')

  return mejor_solucion

In [86]:
#Invocamos a la solución greedy multiarranque

solucion = solucion_greedy_multiarranque( tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')

Iteracion = 0 Costo = 31
Iteracion = 1 Costo = 29
[[ 1 23  2]
 [ 1 17  0]
 [ 1 19  0]
 [ 1 14  1]
 [ 1 18  0]
 [ 1 24  0]
 [ 2  8  3]
 [ 2  9  1]
 [ 2 12  1]
 [ 2 22  0]
 [ 2 28  0]
 [ 2 30  0]
 [ 3  1  5]
 [ 3  2  0]
 [ 3  6  0]
 [ 3  7  0]
 [ 3 13  0]
 [ 3 20  0]
 [ 4 27  2]
 [ 4 29  2]
 [ 4  3  2]
 [ 4 15  0]
 [ 4  4  1]
 [ 4  5  0]
 [ 5 11  5]
 [ 5 21  1]
 [ 5 10  1]
 [ 5 26  0]
 [ 5 16  2]
 [ 5 25  0]]
solucion = 29


#4. Conclusión:
La mejor solución se obtiene con el algoritmo greedy con multiarranque, con un costo de 29.


#5. Bonus.
#### Probamos la solución con en un escenario de 60 tomas y 10 actores generada de forma aleatoria.

In [88]:
# Probamos la solución si tuviérmos un número mayor de tomas
# Para eso creamos una matriz con 1 y 0 de manera aleatoria, con 60 tomas y 10 actores

tomas_actores = np.random.randint(2, size=(60, 10))
solucion = solucion_greedy_multiarranque(tomas_actores)
print (solucion)
print (f'solucion = {np.sum(solucion[:,2], axis = 0)}')


Iteracion = 0 Costo = 89
Iteracion = 1 Costo = 87
Iteracion = 11 Costo = 86
[[ 1 22  4]
 [ 1 35  0]
 [ 1 10  1]
 [ 1 16  0]
 [ 1  8  1]
 [ 1 15  0]
 [ 2 53  3]
 [ 2 58  2]
 [ 2  3  2]
 [ 2 26  0]
 [ 2 45  0]
 [ 2 52  0]
 [ 3 57  4]
 [ 3  1  2]
 [ 3  2  0]
 [ 3 19  1]
 [ 3 20  0]
 [ 3 24  0]
 [ 4 39  4]
 [ 4  4  4]
 [ 4  6  1]
 [ 4 11  0]
 [ 4 13  0]
 [ 4 14  0]
 [ 5 25  4]
 [ 5 28  1]
 [ 5 30  1]
 [ 5 37  1]
 [ 5 48  2]
 [ 5 50  0]
 [ 6 51  4]
 [ 6 55  3]
 [ 6 60  1]
 [ 6  5  1]
 [ 6  7  0]
 [ 6  9  1]
 [ 7 12  6]
 [ 7 17  1]
 [ 7 18  2]
 [ 7 21  0]
 [ 7 23  0]
 [ 7 27  1]
 [ 8 29  6]
 [ 8 31  0]
 [ 8 32  3]
 [ 8 33  0]
 [ 8 34  0]
 [ 8 36  0]
 [ 9 38  5]
 [ 9 40  1]
 [ 9 41  2]
 [ 9 42  0]
 [ 9 43  1]
 [ 9 44  0]
 [10 46  4]
 [10 47  4]
 [10 49  2]
 [10 54  0]
 [10 56  0]
 [10 59  0]]
solucion = 86
